# Kohonen map

### Цель: выделить основные темы (представленные в виде набора слов), которые встречаются в коллекции документов, с помощью MiniSom. 

In [2]:
pip install MiniSom

  Created wheel for MiniSom: filename=MiniSom-2.2.7-py3-none-any.whl size=8613 sha256=ce77a29263cb14f4400603f562b90bf2a505fbea0c62a9d230b5c225febb58f6
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\0e\26\1a\6859e03682075865e482f052bf299f5de86d27fd9891dc2717
Successfully built MiniSom
Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from minisom import MiniSom
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer

Датасет - набор данных 20newsgroups - это коллекция примерно из 20000 новостных документов, разделенная (приблизительно) равномерно между 20 различными категориями. 

https://www.kaggle.com/crawford/20-newsgroups

Загрузим набор данных с помощью sklearn и преобразуем текстовые документы в матрицу D, где каждая строка представляет сообщение с использованием представления TF-IDF.

### TF-IDF
TF-IDF - это  term frequency-inverse document frequency или частотность терминов-обратная частотность документов.

Это простой и удобный способ оценить важность термина для какого-либо документа относительно всех остальных документов. Принцип такой — если слово встречается в каком-либо документе часто, при этом встречаясь редко во всех остальных документах — это слово имеет большую значимость для того самого документа.


Тerm Frequency - TF  — это частотность термина, которая измеряет, насколько часто термин встречается в документе. Логично предположить, что в длинных документах термин может встретиться в больших количествах, чем в коротких, поэтому абсолютные числа тут не катят. Поэтому применяют относительные — делят количество раз, когда нужный термин встретился в тексте, на общее количество слов в тексте.

TF термина а = (Количество раз, когда термин а встретился в тексте / количество всех слов в тексте)

IDF — это обратная частотность документов. Она измеряет непосредственно важность термина. То есть, когда мы считали TF, все термины считаются как бы равными по важности друг другу. Но всем известно, что, например, предлоги встречаются очень часто, хотя практически не влияют на смысл текста. И что с этим поделать? Ответ прост — посчитать IDF. Он считается как логарифм от общего количества документов, делённого на количество документов, в которых встречается термин а.

IDF термина а = логарифм(Общее количество документов / Количество документов, в которых встречается термин а)

In [4]:
dataset = fetch_20newsgroups(shuffle=True, random_state=1,
                             remove=('headers', 'footers', 'quotes'))
documents = dataset.data

no_features = 1000

tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2,
                                   max_features=no_features,
                                   stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(documents)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
D = tfidf.todense().tolist()

Теперь нам нужно обучить SOM, который кластеризует документы, общее количество нейронов в SOM будет также количеством тем для извлечения:

In [5]:
n_neurons = 2 # на вход подается двухмерный вектор
m_neurons = 4 # сеть будет размером 4x4 
# Т.е. каждый нейрон представляет собой n-мерный вектор-столбец, 
# где n определяется размерностью исходного пространства (размерностью входных векторов).
som = MiniSom(n_neurons, m_neurons, no_features)
som.pca_weights_init(D)
som.train(D, 40000, random_order=False, verbose=False)

Мы будем рассматривать в качестве темы список первых десяти top_keywords, связанных с наибольшими весами каждого нейрона. 

С помощью следующего цикла for мы проверим все веса и восстановим слова, связанные с весами, используя имена функций, сохраненные TfidfVectorizer:

In [6]:
top_keywords = 10

weights = som.get_weights()
cnt = 1
for i in range(n_neurons):
    for j in range(m_neurons):
        keywords_idx = np.argsort(weights[i,j,:])[-top_keywords:]
        keywords = ' '.join([tfidf_feature_names[k] for k in keywords_idx])
        print('Topic', cnt, ':', keywords)
        cnt += 1

Topic 1 : low reported think want truth knowledge shall right people don
Topic 2 : don clipper games like armenians people turkey turkish space armenia
Topic 3 : generally statement lots dos true money machines dod os like
Topic 4 : service cards scsi matter 17 know deleted 19 stuff cable
Topic 5 : finally ed email ideas home people didn pick alt learn
Topic 6 : used better mail buy x11 pc info thanks appreciated advance
Topic 7 : light religion administration expect money like man encryption clinton jesus
Topic 8 : mail stuff know jesus card state book thank report mode
